In [ ]:
# =============================================================================
# Part 0: Setup and Library Imports
# =============================================================================
print("--- Part 0: Loading Libraries ---")
# Install dependencies if running for the first time
# !pip install biopython scikit-learn pandas xgboost matplotlib imblearn joblib

from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
import joblib

# Set display option for matplotlib in Jupyter
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
# --- NEW: Import VarianceThreshold ---
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [ ]:
# =============================================================================
# Part 1: Data Collection and Labeling
# =============================================================================
print("\n--- Part 1: Collecting and Labeling Data ---")
gbk_file = "GCF_000012145.1_ASM1214v1_genomic.gbff"
if not os.path.exists(gbk_file):
    print("Downloading and decompressing GenBank file...")
    !wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/012/145/GCF_000012145.1_ASM1214v1/GCF_000012145.1_ASM1214v1_genomic.gbff.gz
    !gunzip GCF_000012145.1_ASM1214v1_genomic.gbff.gz
    print("Download complete.")

stress_genes_positive = [
    "DR2340", "DR1089", "DRA0346", "DR0423", "DR1596", "DR1595", "DR2256", "DR1337", "DR0845", "DR1205", "DR1132",
    "DR0757", "DR1720", "DR0325", "DR0828", "DRA0013", "DRA0014", "DRA0016", "DRA0017", "DR1289", "DR1126", "DR1477",
    "DR1775", "DR1902", "DR0099", "DR0070", "DR1916", "DR2633", "DR0326", "DR0198", "DR0219", "DR1901", "DR2441",
    "DRB0095", "DR1857", "DR0100", "DR0689", "DR0261", "DR2574", "DR1998", "DRB0100", "DR0003", "DR0505", "DR0596",
    "DR1172", "DR0607", "DR0819", "DR2275", "DR2417"
]
housekeeping_genes_negative = [
    "DR0001", "DR0002", "DR0004", "DR0005", "DR0006", "DR0010", "DR1343", "DR1344", "DR0634", "DR0635", "DR1796",
    "DR0012", "DR0013", "DR0014", "DR0015", "DR0020", "DR0021", "DR0022", "DR0907", "DR0908", "DR1913", "DR1914",
    "DR2088", "DR2089", "DR2090", "DR1262", "DR1263", "DR0016", "DR0017", "DR0023", "DR0024", "DR0102", "DR0103",
    "DR1264", "DR1265", "DR2091", "DR2092", "DR0506", "DR0507", "DR0028", "DR0029", "DR0030", "DR0107", "DR0108",
    "DR0109", "DR1268", "DR1269", "DR2095", "DR2096", "DR0508", "DR0509", "DR0110", "DR0111", "DR0112"
]
keywords = [
    "repair", "stress", "tolerance", "resistance", "oxidative", "radiation", "dna", "antioxidant", "heat shock",
    "cold shock", "desiccation", "uv", "gamma", "protection", "survival", "adaptation", "chaperone",
    "detoxification", "redox", "homeostasis", "shock", "survivability", "resilience", "stabilization", "recovery",
    "protective", "oxidoreductase", "peroxidase", "superoxide", "catalase", "stress-induced"
]
labeled_genes = []
records = list(SeqIO.parse(gbk_file, "genbank"))
for record in records:
    for feature in record.features:
        if feature.type == "CDS":
            gene_id = feature.qualifiers.get("locus_tag", [None])[0]
            if not gene_id: continue
            product = feature.qualifiers.get("product", [""])[0].lower()
            sequence = str(feature.extract(record.seq))
            if len(sequence) % 3 != 0 or 'N' in sequence: continue
            label = -1
            if gene_id in stress_genes_positive or any(keyword in product for keyword in keywords):
                label = 1
            elif gene_id in housekeeping_genes_negative or "ribosomal protein" in product or \
                 any(term in product for term in ["elongation factor", "tRNA synthetase", "gyrase"]):
                label = 0
            if label != -1:
                labeled_genes.append({"id": gene_id, "sequence": sequence, "label": label})
df = pd.DataFrame(labeled_genes).drop_duplicates(subset=['id']).reset_index(drop=True)
print("Cleaned Class Distribution:")
print(df['label'].value_counts())


In [ ]:
# =============================================================================
# Part 2: Feature Engineering
# =============================================================================
print("\n--- Part 2: Engineering Features ---")
def get_protein_features(seq):
    try:
        translated_seq = seq.translate(to_stop=True)
        if len(translated_seq) < 10: return 0, 0
        protein_analyzer = ProteinAnalysis(str(translated_seq))
        hydrophobicity = protein_analyzer.gravy()
        isoelectric_point = protein_analyzer.isoelectric_point()
        return hydrophobicity, isoelectric_point
    except Exception:
        return 0, 0
def get_all_features(df):
    basic_features_list = []
    motifs = [r"GTT[ATCG]CG", r"CG[ATCG]CG", r"TTC[ATCG][ATCG]GAA", r"CTG[ATCG][ATCG]GTC"]
    for seq in df['sequence']:
        length = len(seq)
        gc_content = (seq.count('G') + seq.count('C')) / length if length > 0 else 0
        gc3 = sum(1 for i in range(2, len(seq), 3) if seq[i] in ['G', 'C']) / (length // 3) if length >= 3 else 0
        cg_dinucleotide = seq.count("CG") / (length - 1) if length > 1 else 0
        motif_frequencies = [len(re.findall(motif, seq, re.IGNORECASE)) / length if length > 0 else 0 for motif in motifs]
        hydrophobicity, isoelectric_point = get_protein_features(seq)
        basic_features_list.append({
            "length": length, "gc_content": gc_content, "gc3": gc3, "cg_dinucleotide": cg_dinucleotide,
            "motif_frequency_1": motif_frequencies[0], "motif_frequency_2": motif_frequencies[1],
            "motif_frequency_3": motif_frequencies[2], "motif_frequency_4": motif_frequencies[3],
            "hydrophobicity": hydrophobicity, "isoelectric_point": isoelectric_point
        })
    basic_features_df = pd.DataFrame(basic_features_list)
    kmer_vectorizer = CountVectorizer(min_df=0.02, ngram_range=(4, 4))
    kmer_texts = [' '.join([seq[i:i+4] for i in range(len(seq) - 3)]) for seq in df['sequence']]
    kmer_counts = kmer_vectorizer.fit_transform(kmer_texts)
    kmer_features_df = pd.DataFrame(kmer_counts.toarray(), columns=kmer_vectorizer.get_feature_names_out())
    full_feature_df = pd.concat([basic_features_df, kmer_features_df], axis=1)
    return full_feature_df, kmer_vectorizer
X, kmer_vectorizer = get_all_features(df)
y = df['label']
feature_columns = X.columns.tolist()
print(f"Total features engineered: {len(feature_columns)}")


In [ ]:
# =============================================================================
# Part 3: Preprocessing, Feature Selection, and Model Training (IMPROVED)
# =============================================================================
print("\n--- Part 3: Preprocessing and Training ---")
# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- NEW STEP 1: Remove constant features ---
print("Step 3a: Removing constant features...")
vt = VarianceThreshold(threshold=0.0)
X_train_no_const = vt.fit_transform(X_train)
X_test_no_const = vt.transform(X_test)
# Keep track of the remaining column names
columns_after_vt = X_train.columns[vt.get_support()]
print(f"Removed {X_train.shape[1] - X_train_no_const.shape[1]} constant features.")

# --- NEW STEP 2: Select top 100 features from the non-constant set ---
print("Step 3b: Selecting top 100 features...")
selector = SelectKBest(f_classif, k=100)
# Use the non-constant data for fitting
X_train_selected = selector.fit_transform(X_train_no_const, y_train)
X_test_selected = selector.transform(X_test_no_const)
# Keep track of the final selected column names
final_columns = columns_after_vt[selector.get_support()]

# --- NEW STEP 3: Apply SMOTE ---
print("Step 3c: Applying SMOTE to balance training data...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_selected, y_train)

# --- NEW STEP 4: Train model ---
print("Step 3d: Performing GridSearchCV for hyperparameter tuning...")
param_grid = {'n_estimators': [100, 200], 'max_depth': [3, 5], 'learning_rate': [0.05, 0.1]}
model_xgb = XGBClassifier(eval_metric='logloss', random_state=42, use_label_encoder=False)
grid = GridSearchCV(model_xgb, param_grid, cv=3, scoring='f1_macro', n_jobs=-1)
grid.fit(X_train_resampled, y_train_resampled)
final_model = grid.best_estimator_
print("Best hyperparameters found:", grid.best_params_)


In [ ]:
# =============================================================================
# Part 4: Model Evaluation
# =============================================================================
print("\n--- Part 4: Evaluating the Final Model ---")
y_pred = final_model.predict(X_test_selected)
y_proba = final_model.predict_proba(X_test_selected)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Control", "Stress"]))

In [ ]:
# =============================================================================
# Part 5: Saving All Model Artifacts for Future Use (IMPROVED)
# =============================================================================
print("\n--- Part 5: Saving Model and Pipeline Artifacts ---")
joblib.dump(final_model, 'stress_gene_model.joblib')
# --- NEW: Save the variance thresholder ---
joblib.dump(vt, 'variance_thresholder.joblib')
joblib.dump(selector, 'feature_selector.joblib')
joblib.dump(kmer_vectorizer, 'kmer_vectorizer.joblib')
joblib.dump(feature_columns, 'feature_columns.joblib')
print("✅ All artifacts saved successfully.")

In [53]:
# =============================================================================
# Part 6: Loading Artifacts and Predicting on 5 Samples (IMPROVED)
# =============================================================================
print("\n--- Part 6: Loading Artifacts and Demonstrating Prediction ---")
# Load all the components
loaded_model = joblib.load('stress_gene_model.joblib')
# --- NEW: Load the variance thresholder ---
loaded_vt = joblib.load('variance_thresholder.joblib')
loaded_selector = joblib.load('feature_selector.joblib')
loaded_vectorizer = joblib.load('kmer_vectorizer.joblib')
loaded_training_columns = joblib.load('feature_columns.joblib')
print("✅ All artifacts re-loaded successfully.\n")

sample_df = df.sample(n=5, random_state=42)
label_map = {0: "Control", 1: "Stress"}
for index, row in sample_df.iterrows():
    gene_id = row['id']
    true_label_numeric = row['label']
    sequence = row['sequence']
    print(f"--- Predicting for Gene ID: {gene_id} ---")
    print(f"True Label: {label_map[true_label_numeric]}")

    # Replicate the full feature engineering and preprocessing pipeline
    single_gene_df = pd.DataFrame([{"sequence": sequence}])
    basic_features_list = []
    # (Re-using the feature generation logic for simplicity)
    motifs = [r"GTT[ATCG]CG", r"CG[ATCG]CG", r"TTC[ATCG][ATCG]GAA", r"CTG[ATCG][ATCG]GTC"]
    length = len(sequence); gc_content = (sequence.count('G') + sequence.count('C')) / length if length > 0 else 0
    gc3 = sum(1 for i in range(2, len(sequence), 3) if sequence[i] in ['G', 'C']) / (length // 3) if length >= 3 else 0
    cg_dinucleotide = sequence.count("CG") / (length - 1) if length > 1 else 0
    motif_frequencies = [len(re.findall(motif, sequence, re.IGNORECASE)) / length if length > 0 else 0 for motif in motifs]
    hydrophobicity, isoelectric_point = get_protein_features(sequence)
    basic_features_list.append({
        "length": length, "gc_content": gc_content, "gc3": gc3, "cg_dinucleotide": cg_dinucleotide,
        "motif_frequency_1": motif_frequencies[0], "motif_frequency_2": motif_frequencies[1],
        "motif_frequency_3": motif_frequencies[2], "motif_frequency_4": motif_frequencies[3],
        "hydrophobicity": hydrophobicity, "isoelectric_point": isoelectric_point
    })
    basic_features = pd.DataFrame(basic_features_list)
    kmer_text = [' '.join([sequence[i:i+4] for i in range(len(sequence) - 3)])]
    kmer_counts = loaded_vectorizer.transform(kmer_text)
    kmer_features = pd.DataFrame(kmer_counts.toarray(), columns=loaded_vectorizer.get_feature_names_out())
    single_feature_vector = pd.concat([basic_features, kmer_features], axis=1)
    aligned_vector = single_feature_vector.reindex(columns=loaded_training_columns, fill_value=0)

    # --- NEW: Apply the loaded preprocessors in the correct order ---
    # 1. Apply VarianceThreshold
    vector_no_const = loaded_vt.transform(aligned_vector)
    # 2. Apply SelectKBest
    selected_vector = loaded_selector.transform(vector_no_const)

    # Make prediction
    prediction = loaded_model.predict(selected_vector)[0]
    probabilities = loaded_model.predict_proba(selected_vector)[0]

    # Print results
    print(f"Predicted Label: {label_map[prediction]}")
    print(f"Model Confidence -> [P(Control): {probabilities[0]:.2f}, P(Stress): {probabilities[1]:.2f}]")
    if prediction == true_label_numeric: print("✅ Result: CORRECT")
    else: print("❌ Result: INCORRECT")
    print("-" * 45 + "\n")

--- Part 0: Loading Libraries ---

--- Part 1: Collecting and Labeling Data ---
Cleaned Class Distribution:
label
1    126
0     59
Name: count, dtype: int64

--- Part 2: Engineering Features ---
Total features engineered: 10792

--- Part 3: Preprocessing and Training ---
Step 3a: Removing constant features...
Removed 2 constant features.
Step 3b: Selecting top 100 features...
Step 3c: Applying SMOTE to balance training data...
Step 3d: Performing GridSearchCV for hyperparameter tuning...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:13:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best hyperparameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}

--- Part 4: Evaluating the Final Model ---
Accuracy: 0.7297297297297297
ROC AUC Score: 0.7200000000000001

Classification Report:
              precision    recall  f1-score   support

     Control       0.57      0.67      0.62        12
      Stress       0.83      0.76      0.79        25

    accuracy                           0.73        37
   macro avg       0.70      0.71      0.70        37
weighted avg       0.74      0.73      0.73        37


--- Part 5: Saving Model and Pipeline Artifacts ---
✅ All artifacts saved successfully.

--- Part 6: Loading Artifacts and Demonstrating Prediction ---
✅ All artifacts re-loaded successfully.

--- Predicting for Gene ID: RF_RS01055 ---
True Label: Stress
Predicted Label: Stress
Model Confidence -> [P(Control): 0.00, P(Stress): 1.00]
✅ Result: CORRECT
---------------------------------------------

--- Predicting for Gene ID: RF_RS01585 ---
True Label